# `qfit` Quick Start

Tianpu Zhao, Danyang Chen and Jens Koch

This guide will walk you through the basic usage of `qfit` and its features. We have prepared a simple task for you to practice fitting with `qfit`. Don't worry, we've also prepared a detailed guide for you to follow. 

## Background & your task

You are given a **superconducting circuit** composed of a fluxonium qubit and a resonator mode. They are capacitively coupled. The effective circuit is shown below:
<p align="center">
  <img width="300" src="resources/images/example_circuit.png">
</p>  

This circuit is described by the following **Hamiltonian**:
$$
H = H_{\rm res} + H_{\rm qubit} + H_{\rm int}
$$
with the resonator and qubit Hamiltonians and their interactions given by
\begin{align*}
H_{\rm res} &= E_{\rm osc} a^\dagger a \\
H_{\rm qubit} &= 4E_C n^2 - E_J \cos(\varphi+2\pi\Phi_{\rm ext}/\Phi_0) + \frac{1}{2}E_L\varphi^2 \\
H_{\rm int} &= g \frac{-i}{\sqrt{2}l_{\rm osc}} (a-a^\dagger) n
\end{align*}
where $a$ is the annihilation operator of the resonator, $n$ ($\varphi$) is the number (phase) operator of the qubit. $\Phi_{\rm ext}$ is the external flux, and $\Phi_0$ is the flux quantum.  Circuit parameters and their estimates (typically obtained from EM simulations and the Ambegaokar–Baratoff relation) are listed below:

| Parameter | Symbol | Rough Estimate |
| --- | --- | --- |
| Josephson Energy | $E_J$ | $3.0\textrm{ GHz}\times h$ |
| Charging Energy | $E_C$ | $0.9\textrm{ GHz}\times h$ |
| Inductive Energy | $E_L$ | $0.25\textrm{ GHz}\times h$ |
| Oscillator Length | $l_{\rm osc}$ | $1.0$ |
| Resonator Frequency | $E_{\mathrm{osc}}$ | $5.6\textrm{ GHz}\times h$ |
| Coupling Strength | $g$ | $1.0 \textrm{ GHz}\times h$ |

To determine the parameter described above, one can perform a **two-tone spectroscopy experiment**. It's summarized in the image below:
<figure align="center">
  <img width="700" src="resources/images/example_two_tone_setup.png">
</figure>  

In the experiment, two tones of RF drive are applied to the system - a tone with frequency $\omega$ to the qubit and a tone near frequency $E_{\rm osc} / h$ to the resonator. One then measure the transmission or reflection of the RF signal through the resonator. If the qubit drive pulse frequency $\omega$ happens to match a transition frequency of the qubit-resonator system, then the pulse may drive this transition. This may introduce a shift on the resonator frequency, which changes the amplitude and phase of the transmission/reflection signal. By performing such measurement for different $\omega$ and $\Phi_{\rm ext}$, one can obtain obtain the spectroscopy data.

Check out below for an example of such data:

In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

# Load the data in the h5 file (you don't need to do this while using QFit)
print("Below is the information of the data and the shape of elements:")
with h5py.File("./example_data/joint_qubit_twotone.h5", "r") as file:
    # Print the information of the data
    def print_info(name, obj):
        print(name, obj.shape)
    file.visititems(print_info)

    # store the data in those variables
    data = np.array(file["mags"])
    omega, voltage = np.meshgrid(file["freq"], file["voltage"], indexing="ij")

# Plot the data
plt.figure(figsize=(7, 5.5))
plt.pcolormesh(voltage, omega, data.T, cmap="PuOr", rasterized=True)
plt.colorbar()
plt.xlabel("Voltage to the flux bias line [mV]")
plt.ylabel("Drive frequency [GHz]")
plt.title("Transmission magnitude")
plt.show()

Below is the information of the data and the shape of elements:
freq (2793,)
mags (81, 2793)
voltage (81,)


In the data, the peaks (appears as dark purple/dark yellow spots) represent resonances between the $\omega$ and the transition frequencies. **Your task is to fit the data and extract the parameters.** - This is where `qfit` comes in. 

## Step 0: Build a numerical model and run `qfit`

We have helped you to create a numerical model of the quantum system using `scqubits.HilbertSpace`. To see the usage of `scqubits` in detail, please refer to the [scqubits documentation](https://scqubits.readthedocs.io/en/latest/).

In [3]:
import scqubits as scq

fluxonium = scq.Fluxonium(
    EJ = 3.0,
    EC = 0.9,
    EL = 0.25,
    flux = 0.5,
    cutoff = 100,
    truncated_dim = 5,
    id_str = "Fluxonium"
)
resonator = scq.Oscillator(
    E_osc = 5.6,
    l_osc = 1.0,
    truncated_dim = 4,
    id_str = "Resonator"
)

hilbert_space = scq.HilbertSpace([fluxonium, resonator])

hilbert_space.add_interaction(
    g = 1,
    op1 = fluxonium.n_operator,
    op2 = resonator.n_operator,
    add_hc = False,
    id_str = "res-qubit"
)

#### Provide experimental data and launch `qfit`

Open the app using `Fit(<HilbertSpace>, <data file path>)`. An app (the graphical user interface, GUI) will pop up and we'll guide you through the GUI in the next few subsections. You may treat the following sections as a documentation and refer to it if necessary. Also you may click on the ? buttons for help.

In [4]:
from qfit import Fit

file_path = "./example_data/joint_qubit_twotone_unif.h5"
fit = Fit(hilbert_space, file_path)

In [5]:
edge_pts = fit._canvas.canvas.figure.bbox.width, fit._canvas.canvas.figure.bbox.height

origin = fit._canvas.axes.transAxes.inverted().transform((0, 0))
edge = fit._canvas.axes.transAxes.inverted().transform(edge_pts)

print(origin, edge, edge_pts)

[-0.19266055 -0.04444444] [1.02293578 1.01010101] (1044.0, 530.0)


In [4]:
fit._canvas.canvas.figure.bbox.__dir__()

['_parents',
 '_invalid',
 '_shorthand_name',
 '_bbox',
 '_transform',
 '_points',
 '__module__',
 '__doc__',
 '__init__',
 '__str__',
 'get_points',
 'is_bbox',
 'is_affine',
 'frozen',
 '__array__',
 'x0',
 'y0',
 'x1',
 'y1',
 'p0',
 'p1',
 'xmin',
 'ymin',
 'xmax',
 'ymax',
 'min',
 'max',
 'intervalx',
 'intervaly',
 'width',
 'height',
 'size',
 'bounds',
 'extents',
 'containsx',
 'containsy',
 'contains',
 'overlaps',
 'fully_containsx',
 'fully_containsy',
 'fully_contains',
 'fully_overlaps',
 'transformed',
 'coefs',
 'anchored',
 'shrunk',
 'shrunk_to_aspect',
 'splitx',
 'splity',
 'count_contains',
 'count_overlaps',
 'expanded',
 'padded',
 'translated',
 'corners',
 'rotated',
 'union',
 'intersection',
 'INVALID_NON_AFFINE',
 'INVALID_AFFINE',
 'INVALID',
 'pass_through',
 '__getstate__',
 '__setstate__',
 '__copy__',
 'invalidate',
 '_invalidate_internal',
 'set_children',
 '__dict__',
 '__weakref__',
 '__new__',
 '__repr__',
 '__hash__',
 '__getattribute__',
 '__seta

In [18]:
fit._canvas.canvas.figure.bbox.size

array([1310., 1060.])

In [25]:
edge_pts = fit._canvas.canvas.figure.bbox.size

origin = fit._canvas.axes.transAxes.inverted().transform((0, 0))
edge = fit._canvas.axes.transAxes.inverted().transform(edge_pts)

print(origin, edge, edge_pts)

[-0.06907895 -0.04373757] [1.00822368 1.00994036] [1310. 1060.]


In [30]:
fit._canvas.canvas.figure.get_size_inches()

array([6.55, 5.3 ])

In [34]:
fit._canvas.canvas.figure.dpi

200.0

In [26]:
edge_pts = fit._canvas.canvas.figure.bbox.size

origin = fit._canvas.axes.transAxes.inverted().transform((0, 0))
edge = fit._canvas.axes.transAxes.inverted().transform(edge_pts)

print(origin, edge, edge_pts)

[-0.14973262 -0.09243697] [1.01782531 1.0210084 ] [1310. 1060.]


In [23]:
edge_pts = fit._canvas.canvas.figure.bbox.width, fit._canvas.canvas.figure.bbox.height

a = fit._canvas.axes.transAxes.inverted().transform((0, 0))
b = fit._canvas.axes.transAxes.inverted().transform((10, 20))

print(a, b, a - b)

[-0.14973262 -0.09243697] [-0.14081996 -0.07142857] [-0.00891266 -0.0210084 ]


In [24]:
edge_pts = fit._canvas.canvas.figure.bbox.width, fit._canvas.canvas.figure.bbox.height

a = fit._canvas.axes.transAxes.inverted().transform((0, 0))
b = fit._canvas.axes.transAxes.inverted().transform((10, 20))

print(a, b, a - b)

[-0.06907895 -0.04373757] [-0.06085526 -0.02385686] [-0.00822368 -0.01988072]


#### An overview of the GUI
After launching the GUI, you will see a window below. The window is divided into parts as shown here. 
<p align="center">
  <img width="600" src="resources/images/example_GUI_overview.png">
</p>  

The fitting task is divided into 5 steps and you can navigate through them using the navigation menu: **Import**, **Calibrate**, **Extract**, **Pre-Fit** and **Fit**. Let's go through them one by one and you may perform the corresponding task as you read along.

## Step 1: Import data

Once the file is imported, `QFit` parses what is contained in the data file. Under <span style="color:rgb(190, 130, 250);">METADATA</span>, you may find the metadata of the imported file. What remains to be completed by you is to <span style="color:rgb(190, 130, 250);">SELECT AXES</span> and thereby adjust orientation of the figure, using the panel below.
<p align="center">
  <img width="300" src="resources/images/example_axes_select.png">
</p> 

Once you select the desired x- and y-axis, the figure will be oriented correctly. And you can proceed to the next step by clicking "Proceed to calibrate" button.

## Step 2: Calibrate

#### The calibration process

In this step, we aim to
1. map the **x-axis** of measurement data to the circuit parameter. In our case, we want to map the flux bias line voltage in mV the external flux in $\Phi_0$. The same principle can also apply to circuits that are tuned by offset charges.
2. map the **y-axis** of measurement data to frequencies **in unit of GHz**, which is `scqubit`'s default unit.

`QFit` assumes a linear relationship between the flux bias line voltage ($x$) and external flux ($x'$), i.e. $x' = ax + b$. In this case, two pairs of $(x, x')$ values to determine the map, as shown below.
<p align="center">
    <img width="350" src="resources/images/calibration_notebook_ver.png">
</p>  


#### Calibration in `qfit`: 
1. Click the "EXTRACT RAW" button. It activates the extraction for $x_1$. 
2. In the plot, identify/guess a zero-flux point and click. The $x_1$ coordinate will be automatically extracted.  
3. Enter the value $x_1^\prime$, which is 0.0 flux quantum.  
4. Repeat the above steps for $x_2$.  
    
You can also check out the video below for a quick demonstration.

<div style="text-align: center;">
  <video src="resources/videos/calibration.mp4" width="500" controls>
    [Your browser does not support the video tag.]
  </video>
</div>

Finally, you can re-plot and check the calibrated axes by clicking "View Calibrated Axes" on the top right corner.

**Tips**

For a fluxonium, the Hamiltonian is periodic in flux with period $\Phi_0$ and symmetric around $\Phi_{\rm ext} = \frac{m}{2} \Phi_0$, where $m$ is an integer. Therefore, we can identify and map the x coordinates with those properties.

## Step 3: Extract

Now let's navigate to the next step and extract resonance peaks out of the spectroscopy data. In `QFit`, extracted data are grouped by  <span style="color:rgb(190, 130, 250);">TRANSITIONS</span>. For example, there is a spectral line in the data which is believed to be the plasmon transition $|\overline{0_{\rm res},0_{\rm qubit}}\rangle \rightarrow|\overline{0_{\rm res},2_{\rm qubit}}\rangle$. Follow these steps to extract and label these data points:

1. Select / create a group of transition data and label it. In this case, we uses bare state label $(0, 0) \to (0, 2)$:
<p align="center">
    <img width="200" src="resources/images/example_transition_n_label.png">
</p>  
   
2. Click on the canvas near a peak to select it. There a few helpful tools for you in this step (see tips below). Here is how it looks.
<p align="center">
    <img width="250" src="resources/images/example_extract02.png">
</p>  

3. If you want to increase the fitting accuracy, extract more transitions as shown below by repeating step 1, 2.
<p align="center">
    <img width="250" src="resources/images/example_transitions.png">
</p>  

**Tips**

To assist you in the data extraction, the following button will be useful when playing with the canvas.
<p align="center">
    <img width="500" src="resources/images/example_MPL_buttons.png">
</p>  

They are (from left to right): **Reset | Pan, Zoom, Extract | X-Snap, Y-Snap | View Calibrated Axes**. 

1. **Pan**, **Zoom** and **Extract** buttons controls the usage of the cursor.
2. **X-Snap** button helps to align the x-coordinate among different transitions (groups). This tool help you align extracted data along the same flux points; by selecting fewer distinct flux points, it speeds up calculations for transition frequencies.

<div style="display: flex; justify-content: center; align-items: center;">
    <img width="200" src="resources/gifs/X_snap.gif">
    <div style="width: 50px;"></div>  <!-- Spacer -->
    <img width="223" src="resources/images/example_same_x_preferred.png">
</div>

3. And the **Y-Snap** button assists you to locate the peak precisely.
<p align="center">
    <img width="200" src="resources/gifs/Y_snap.gif">
</p>

4. To **delete points**, select the corresponding transition and click on the vicinity of the extracted point.


## Step 4: Pre-Fit

After getting enough data, we now start to fit by eye, aiming to find a good initial guess for the numerical fitting. In this step, you may play with **sliders** to make the numerical spectrum match the data. During prefit, numerical spectrum will be calculated and plotted on top of the measurement data. Besides, status bar will be updated with current Mean Squared Error (MSE), indicating how close the fit is.

<p align="center">
    <img height="20" src="resources/images/example_prefit_status.png">
</p>

You may also fine-tune your calibration by adjusting the corresponding sliders, which controls the value that the raw axes are mapped to.


After getting a good set of parameters, you will click **Result To Fit** button on the bottom left to copy the parameters to the <span style="color:rgb(190, 130, 250);">FIT</span> panel.

## Step 5: Fit
Finally, we are ready to fit the data numerically! The main thing you need to do is configure the fitting parameters in the **FIT table**. You may adjust the following entries in the table:
- Whether to **Fix** or free a parameter during fitting
- The **Initial** guesses (from pre-fit)
- **Min** and **Max** values

Now click **Run Fit** and wait for the fitting to finish. 

If there is not warning or error message, congratulations! You have successfully fit the data and the fit parameters can be found in **Current** column. 

If you want to make use of the fitting result as a initial guess, and repeat the numerical fitting with different configurations, you may click **Result To Initial**. In addition, clicking **Result To Prefit** will send the fit result data to the <span style="color:rgb(190, 130, 250);">PRE-FIT</span> panel.

## Save, load and export
You may save / load the fitting session by clicking the <img width="20" src="resources/images/example_menu.png"> button on the top left. The session will be saved as a `.qfit` file.

You can also open a new window to load the previous session.

In [1]:
from qfit import Fit
# this is an example file we already prepared for you
# If you didn't close the previous window, you'll get another one
fit = Fit.open("example_data/QFit_Quick_Start.qfit")

qt.qpa.fonts: Populating font family aliases took 133 ms. Replace uses of missing font family "Roboto Medium" with one that exists to avoid this cost. 


Traceback (most recent call last):
  File "/Users/pacosynthesis/Desktop/ScienceTech/Research/Superconducting_qubit_NU/qfit/qfit/qfit/controllers/io_ctrl.py", line 433, in closeByMainWindow
    status = self.closeAppAfterSaving()
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pacosynthesis/Desktop/ScienceTech/Research/Superconducting_qubit_NU/qfit/qfit/qfit/controllers/io_ctrl.py", line 267, in closeAppAfterSaving
    assert registryDictFromFile is not None, "File not found"
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: File not found


There is another option for you to continue working in the notebook - export.

In [ ]:
# To get the HilbertSpace object and continue working on it
fit_hilbertspace = fit.exportHilbertSpace(deepcopy=True, fromFit=True)
fit_hilbertspace

In [ ]:
# To get the parameters for the HilbertSpace and calibration
fit_parameters = fit.exportParameters(fromFit=True)
fit_parameters

## Final words
We hope you have enjoyed this tutorial and learned how to use `QFit`. There are a few advanved tips in `QFit_Advanced_Tips.ipynb`, please check out if you want to make full use of the package. 

If you have any questions, please feel free to contact us by:  
Tianpu Zhao: TianpuZhao2022@u.northwestern.edu  
Danyang Chen: DanyangChen2026@u.northwestern.edu